# Import / Preprocessing

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing

In [2]:
# Read in the datas
training = pd.read_csv("../dat/training_encoded_scaled.csv")
evaluation = pd.read_csv("../dat/evaluatation_encoded_scaled.csv")

In [3]:
def splitDF(df):
    # create a random permutation of indices
    indices = np.random.permutation(df.shape[0])

    # split permutation in two
    numInd = len(indices)
    trainingInds = indices[:math.ceil(numInd/2)]

    # split df using permutations
    inds = df.index.isin(trainingInds)
    training = df[inds]
    testing = df[~inds]

    # double check shape
    print(testing.shape)
    print(training.shape)
    
    return (testing, training)

In [4]:
# Make a training and testing set
train, test = splitDF(training.drop("id", axis=1))

(94159, 131)
(94159, 131)


## Gradient boost with PCA

In [5]:
from sklearn.decomposition import PCA

In [6]:
train.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.432954,0.394501,0.487975,0.352841,0.441406,0.314495,0.343023,0.618460,0.189002,1283.60
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.168119,0.191194,0.011744,0.221848,0.213371,0.175796,0.172492,0.248701,0.381278,2763.85
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.357327,0.356432,0.042350,0.464353,0.508111,0.344052,0.335689,0.349116,0.822813,5142.87
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.867807,0.552270,0.892576,0.852379,0.806227,0.631803,0.778875,0.869742,0.095828,3585.75
10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.423270,0.868785,0.211377,0.538088,0.508111,0.534769,0.521943,0.340023,0.805128,6396.85


In [7]:
X = train.drop("loss", axis=1)
X_t = test.drop("loss", axis=1)
y = train["loss"]
y_t = test["loss"]

In [8]:
pca = PCA(n_components = 65, svd_solver = 'full')
pca.fit(X)
X_PCA = pca.transform(X)
X_t_PCA = pca.transform(X_t)
print(X_PCA.shape)
print(X_t_PCA.shape)

(94159, 65)
(94159, 65)


In [14]:
import sklearn.ensemble as em
m = em.GradientBoostingRegressor(max_depth=10)
m.fit(X, y)

KeyboardInterrupt: 

In [ ]:
y_hat = m.predict(X_t)

In [ ]:
def score(y, y_hat):
    return abs(y - y_hat).mean()

In [ ]:
score(y_hat, y_t)

In [12]:
evaluation["loss"] = y_hat
evaluation[["id","loss"]].to_csv("./submission11.csv", index=False)

ValueError: Length of values does not match length of index

In [ ]:
X_t_PCA.shape

In [ ]:
X_t.shape

## TODOs
- run xgboost
- read about effective dimensionality reduction
- read about scaling for boosting algos